In [1]:
#import all the dependencies
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import  matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from skimage.feature import hog

ImportError: No module named 'skimage'